In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-227082
Azure region: eastus2
Subscription id: f5091c60-1c3c-430f-8d81-d802f6bf2414
Resource group: aml-quickstarts-227082


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "udacity-project-one-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    aml_compute = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [8]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform, normal
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
### YOUR CODE HERE ###
ps = RandomParameterSampling({"batch_size": choice(16, 32, 64, 128), 
                               "learning_rate": normal(10, 3), 
                               "keep_probability": uniform(0.05, 0.1)})



# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      arguments=['--C', 1.0, '--max_iter', 100],
                      compute_target=aml_compute,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [9]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config)

In [10]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n learning rate:',parameter_values[3])
print('\n keep probability:',parameter_values[5])
print('\n batch size:',parameter_values[7])

AttributeError: module 'numba' has no attribute 'core'

# AutoML Part

In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
df = TabularDatasetFactory().from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")


In [4]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(df)

In [5]:
from azureml.train.automl.utilities import get_primary_metrics
get_primary_metrics('classification')

['norm_macro_recall',
 'AUC_weighted',
 'accuracy',
 'precision_score_weighted',
 'average_precision_score_weighted']

In [10]:
import pandas as pd
pd.concat([x, y]).rename({0:'y'}, axis='columns')

,y,age,campaign,cons.conf.idx,cons.price.idx,contact_cellular,contact_telephone,day_of_week,default,duration,...,job_technician,job_unemployed,job_unknown,loan,marital,month,nr.employed,pdays,poutcome,previous
0,NaN,57.0,1.0,-46.2,92.893,1.0,0.0,1.0,no,371.0,...,1.0,0.0,0.0,1.0,1.0,5.0,5099.1,999.0,0.0,1.0
1,NaN,55.0,2.0,-36.4,93.994,0.0,1.0,4.0,unknown,285.0,...,0.0,0.0,1.0,0.0,1.0,5.0,5191.0,999.0,0.0,0.0
2,NaN,33.0,1.0,-46.2,92.893,1.0,0.0,5.0,no,52.0,...,0.0,0.0,0.0,0.0,1.0,5.0,5099.1,999.0,0.0,1.0
3,NaN,36.0,4.0,-41.8,94.465,0.0,1.0,5.0,no,355.0,...,0.0,0.0,0.0,0.0,1.0,6.0,5228.1,999.0,0.0,0.0
4,NaN,27.0,2.0,-42.7,93.918,1.0,0.0,5.0,no,189.0,...,0.0,0.0,0.0,0.0,1.0,7.0,5228.1,999.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32946,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32947,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32948,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.


automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    X=x,
    y=y,
    training_data=pd.concat([x, y]).rename({0:'y'}, axis='columns'),
    iterations=30,
    label_column_name='y',
    n_cross_validations=5)

In [12]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_output=True)

ConfigException: ConfigException:
	Message: Conflicting or duplicate values are provided for arguments: [label_column_name, weight_column_name, cv_split_column_names]
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Conflicting or duplicate values are provided for arguments: [label_column_name, weight_column_name, cv_split_column_names]",
        "details_uri": "https://aka.ms/AutoMLConfig",
        "target": "X/y & training_data/validation_data",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentMismatch",
                "inner_error": {
                    "code": "ConflictingValueForArguments"
                }
            }
        }
    }
}

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)
assert(automl_run.get_status() == "Completed")

best_run = automl_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n learning rate:',parameter_values[3])
print('\n keep probability:',parameter_values[5])
print('\n batch size:',parameter_values[7])

# Cleanup

In [ ]:
ws.delete(delete_dependent_resources=True, no_wait=False)

In [ ]:
aml_compute.delete()